In [1]:
from medcat.cat import CAT
import json

/home/k1897038/anaconda3/envs/medcat/lib/python3.8/site-packages/medcat/cat.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/home/k1897038/anaconda3/envs/medcat/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
cat = CAT.load_model_pack('/data/users/k1897038/mc_modelpack_phase2_snomed_190k_october_2021.zip')

Found an existing unziped model pack at: /data/users/k1897038/mc_modelpack_phase2_snomed_190k_october_2021, the provided zip will not be touched.


In [ ]:
list(cat.cdb.cui2type_ids.values())[0]

In [82]:
# problem types i.e.  disorders, findings 
problem_types = ['T-11', 'T-18', 'T-29', 'T-35', 'T-38']
# # descriptions of problem, i.e. locations, temperature, respiratory rate.
# descriptors = ['T-6']
# interventions, i.e. investigative scans, procedures, drugs 
interventions = ['T-9', 'T-26', 'T-27', 'T-39', 'T-40', 'T-55']

In [83]:
for p in problem_types:
    print(f'{p}\t{cat.cdb.addl_info["type_id2name"][p]}\t{len(cat.cdb.addl_info["type_id2cuis"][p])}')    

T-11	disorder	77284
T-18	finding	44201
T-29	morphologic abnormality	4897
T-35	organism	34778
T-38	physical object	198890


In [84]:
for i in interventions:
    print(f'{i}\t{cat.cdb.addl_info["type_id2name"][i]}\t{len(cat.cdb.addl_info["type_id2cuis"][i])}')

T-9	clinical drug	6247
T-26	medicinal product	7715
T-27	medicinal product form	6203
T-39	procedure	64291
T-40	product	173894
T-55	substance	27626


In [8]:
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset, load_metric

In [9]:
fl = json.loads(open('../../hadms_to_dis_course_processed.jsonl').readline())

In [10]:
fl = fl['ents']['entities']

In [29]:
next(iter(fl['ents']['entities'].values()))['meta_anns'].values()

dict_values([{'value': 'Patient', 'confidence': 1.0, 'name': 'Subject'}, {'value': 'True', 'confidence': 0.9999957084655762, 'name': 'Presence'}, {'value': 'Recent', 'confidence': 0.9999966025352478, 'name': 'Time'}])

In [ ]:
anns = next(iter(fl.values()))

In [85]:
all_types = problem_types + interventions

In [ ]:
json.loads(open('../../hadms_to_dis_course_processed.jsonl').readline())['text']
# ../../hadms_to_hosp_course_processed.jsonl

In [8]:
def process_cat_out(file_path: str):
    ents = {'hadm_id': [], 'ents': [], 'texts': []}
    for line in tqdm(open(file_path).readlines()):
        line = json.loads(line)
        ents['hadm_id'].append(line['hadm_id'])
        ents['texts'].append(line['text'])
        found_ents = []
        for e in line['ents']['entities'].values():
            anns = e['meta_anns']
            if anns['Subject']['value'] == 'Patient' and anns['Presence']['value'] == 'True' and anns['Time']['value'] == 'Recent':
                if any(t in all_types for t in e['type_ids']):
                    found_ents.append({'pretty_name': e['pretty_name'], 'cui': e['cui'], 
                                       'source_value': e['source_value'], 'types': e['type_ids']})
        ents['ents'].append(found_ents)
    return ents

In [9]:
dis_course_ents = pd.DataFrame(process_cat_out('../../hadms_to_dis_course_processed.jsonl'))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47900/47900 [00:17<00:00, 2701.81it/s]


In [10]:
dis_course_ents = pd.DataFrame(dis_course_ents)

In [11]:
np.average(dis_course_ents.ents.apply(len))

19.764363256784968

In [12]:
uniq_dis_avg_terms = dis_course_ents['ents'].apply(lambda entities: len(set(e['cui'] for e in entities)))
np.average(uniq_dis_avg_terms)

14.577745302713987

In [13]:
notes_ents = process_cat_out('../../hadms_to_hosp_course_processed.jsonl')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47900/47900 [03:32<00:00, 225.09it/s]


In [14]:
notes_ents = pd.DataFrame(notes_ents)

In [15]:
np.average(notes_ents['ents'].apply(len))

156.17847599164926

In [16]:
uniq_avg_terms = notes_ents['ents'].apply(lambda entities: len(set(e['cui'] for e in entities)))
np.average(uniq_avg_terms)

56.62229645093946

In [10]:
model = 't5-small'

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)

In [ ]:
input_ids_hosp_notes_lens = [len(i) for i in input_ids_hosp_notes]
input_ids_dis_course_lens = [len(i) for i in input_ids_dis_course]

In [103]:
np.average([h_n / n for h_n, n in zip(input_ids_hosp_notes_lens, 
                                      notes_ents['ents'].apply(len))])

np.average([h_n / n for h_n, n in zip(input_ids_hosp_notes_lens, uniq_avg_terms)])

In [ ]:
dis_course_ents

In [105]:
np.average([h_n / n for h_n, n in zip(input_ids_dis_course_lens, 
                                      dis_course_ents['ents'].apply(len)) if n > 0])

52.20862549510779

In [115]:
np.average([h_n / n for h_n, n in zip(input_ids_dis_course_lens, 
                                      uniq_dis_avg_terms) if n > 0])

63.10797077817343